In [1]:
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# get absolute path in jupyter notebook
cwd = os.path.abspath("")
images_path = "C:/Users/marti/Documents/courses/Traco/data/dataset/new/images"#os.path.abspath(os.path.join(cwd, os.pardir, "new_data", "my", "images"))
labels_path = "C:/Users/marti/Documents/courses/Traco/data/dataset/new/labels"#os.path.abspath(os.path.join(cwd, os.pardir, "new_data", "my", "labels"))
# take all csv files from labels folder
tracking_labels_files = [os.path.abspath(os.path.join(labels_path, f)) for f in os.listdir(labels_path) if f.endswith('.csv')]
print(f"{tracking_labels_files}")

# Visualize Yolo Format

In [ ]:
# Function to convert YOLO format to bounding box coordinates
def yolo_to_bbox(yolo_bbox, img_width, img_height):
    class_id, x_center, y_center, width, height = yolo_bbox
    x_center, y_center, width, height = float(x_center), float(y_center), float(width), float(height)
    
    x_min = int((x_center - width / 2) * img_width)
    x_max = int((x_center + width / 2) * img_width)
    y_min = int((y_center - height / 2) * img_height)
    y_max = int((y_center + height / 2) * img_height)
    
    return class_id, x_min, y_min, x_max, y_max

# Get a list of image files
image_files = [f for f in os.listdir(images_path) if f.endswith('.jpg')]
print(image_files)
for image_file in image_files:
    # Get the corresponding label file
    label_file = image_file.replace('.jpg', '.txt')
    label_path = os.path.join(labels_path, label_file)
    print(label_path)
    image_path = os.path.join(images_path, image_file)
    print(image_path)
    print()
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        print('Could not read image:', image_file)
        continue
    img_height, img_width, _ = image.shape

    # Read the label file
    with open(label_path, 'r') as file:
        lines = file.readlines()

    # Parse the bounding boxes and draw them on the image
    for line in lines:
        yolo_bbox = line.strip().split()
        bbox = yolo_to_bbox(yolo_bbox, img_width, img_height)
        class_id, x_min, y_min, x_max, y_max = bbox
        
        # Draw the bounding box
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)
        cv2.putText(image, str(class_id), (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    
    # Display the image
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(image_file)
    plt.axis('off')
    plt.show()


# Visualize CSV Format

In [ ]:

# Function to plot points on images
def plot_points_on_image(image, points):
    for point in points:
        frame_id, object_id, x, y = point
        cv2.circle(image, (int(x), int(y)), 5, (0, 255, 0), -1)
        cv2.putText(image, str(object_id), (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    return image

# Get a list of image files
image_files = [f for f in os.listdir(images_path) if f.endswith('.jpg')]

# Process each CSV file
for label_file in os.listdir(labels_path):
    if label_file.endswith('.csv'):
        base_name = label_file.split('.')[0]
        label_path = os.path.join(labels_path, label_file)
        
        # Read the CSV file
        df = pd.read_csv(label_path)
        
        # Group by frame id
        grouped = df.groupby('t')
        
        for frame_id, group in grouped:
            frame_str = f"{base_name}_{frame_id:03d}.jpg"
            if frame_str in image_files:
                image_path = os.path.join(images_path, frame_str)
                image = cv2.imread(image_path)
                
                # Extract points for the current frame
                points = group[['t', 'hexbug', 'x', 'y']].values.tolist()
                
                # Plot points on the image
                image_with_points = plot_points_on_image(image, points)
                
                # Display the image
                plt.figure(figsize=(10, 10))
                plt.imshow(cv2.cvtColor(image_with_points, cv2.COLOR_BGR2RGB))
                plt.title(frame_str)
                plt.axis('off')
                plt.show()


In [ ]:
C:\Users\marti\Documents\courses\Traco\new_data\my\labels